In [4]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest

In [5]:
import os
import base64  

In [6]:
endpoint = "https://nomuratest-doc.cognitiveservices.azure.com/"
key = "94FJpFnAwdNkjiwnCc1b0FY8kOEq2aaQsYUoHlUOyjtnvwfwVaL4JQQJ99BBACHYHv6XJ3w3AAALACOGbxDi"

In [7]:
model_id = "0318_CTBC"

In [8]:
document_intelligence_client  = DocumentIntelligenceClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)

In [9]:
src_path = r'D:\.azure-search-openai-demo_repo\個股資訊 Pdf 文件\整理過的\元大'
pdf_file = r'穎崴 (6515 TT，買進) - 20250206.pdf'
pdf_src = os.path.join(src_path, pdf_file)

In [10]:
with open(pdf_src, "rb") as f:
    base64_encoded_pdf = base64.b64encode(f.read()).decode("utf-8")

In [11]:
# Make sure your document's type is included in the list of document types the custom model can analyze
poller = document_intelligence_client.begin_analyze_document(
    model_id, AnalyzeDocumentRequest(bytes_source=base64_encoded_pdf)
)

In [12]:
result = poller.result()

In [13]:
pdf_base, pdf_ext = os.path.splitext(pdf_file)

In [14]:
with open(os.path.join(src_path, f'output_{pdf_base}.txt'), 'w', encoding='utf-8') as fout:
    for idx, document in enumerate(result.documents):
        print( "--------Analyzing document #{}--------".format(idx + 1), file=fout)
        print( "Document has type {}".format(document.doc_type), file=fout)
        print( "Document has confidence {}".format(document.confidence), file=fout)
        print( "Document was analyzed by model with ID {}".format(result.model_id), file=fout)
        for name, field in document.fields.items():
            print( "......found field of type '{}' with value '{}' and with confidence {}".format(field.type, field.content, field.confidence), file=fout)
    print("-----------------------------------", file=fout)

In [16]:
document.fields

{'broker_name': {'type': 'string', 'valueString': '穎崴\nWinWay', 'content': '穎崴\nWinWay', 'boundingRegions': [{'pageNumber': 1, 'polygon': [0.5127, 0.8206, 3.8829, 0.8721, 3.8772, 1.2466, 0.5069, 1.1951]}], 'confidence': 0.44, 'spans': [{'offset': 25, 'length': 2}, {'offset': 37, 'length': 6}]},
 'content': {'type': 'string', 'valueString': '資料來源:公司資料、元大投顧\n請參閱末頁之免責宣言。', 'content': '資料來源:公司資料、元大投顧\n請參閱末頁之免責宣言。', 'boundingRegions': [{'pageNumber': 3, 'polygon': [0.4882, 8.6747, 3.2496, 8.6784, 3.2459, 11.4429, 0.4845, 11.4392]}], 'confidence': 0, 'spans': [{'offset': 4704, 'length': 26}]},
 'report_date': {'type': 'string', 'valueString': '2025年2月6日', 'content': '2025年2月6日', 'boundingRegions': [{'pageNumber': 1, 'polygon': [5.6214, 0.2155, 6.6449, 0.2179, 6.6445, 0.3796, 5.621, 0.3773]}], 'confidence': 0.49, 'spans': [{'offset': 5, 'length': 9}]},
 'content3': {'type': 'string', 'valueString': '-35.6%\n-22.5%\n後\n404\n-27.1%\n406.8%\n457\n-35.5%', 'content': '-35.6%\n-22.5%\n後\n404\n-27.

In [17]:
for i, j in document.fields.items():
    print(i, j)

broker_name {'type': 'string', 'valueString': '穎崴\nWinWay', 'content': '穎崴\nWinWay', 'boundingRegions': [{'pageNumber': 1, 'polygon': [0.5127, 0.8206, 3.8829, 0.8721, 3.8772, 1.2466, 0.5069, 1.1951]}], 'confidence': 0.44, 'spans': [{'offset': 25, 'length': 2}, {'offset': 37, 'length': 6}]}
content {'type': 'string', 'valueString': '資料來源:公司資料、元大投顧\n請參閱末頁之免責宣言。', 'content': '資料來源:公司資料、元大投顧\n請參閱末頁之免責宣言。', 'boundingRegions': [{'pageNumber': 3, 'polygon': [0.4882, 8.6747, 3.2496, 8.6784, 3.2459, 11.4429, 0.4845, 11.4392]}], 'confidence': 0, 'spans': [{'offset': 4704, 'length': 26}]}
report_date {'type': 'string', 'valueString': '2025年2月6日', 'content': '2025年2月6日', 'boundingRegions': [{'pageNumber': 1, 'polygon': [5.6214, 0.2155, 6.6449, 0.2179, 6.6445, 0.3796, 5.621, 0.3773]}], 'confidence': 0.49, 'spans': [{'offset': 5, 'length': 9}]}
content3 {'type': 'string', 'valueString': '-35.6%\n-22.5%\n後\n404\n-27.1%\n406.8%\n457\n-35.5%', 'content': '-35.6%\n-22.5%\n後\n404\n-27.1%\n406.8%\n457\n-3

In [18]:
import json

In [22]:
with open(os.path.join(src_path, f'output_{pdf_base}.json'), 'w', encoding='utf-8') as fp:
    json.dump(document.fields, fp, indent=4, ensure_ascii=False)

TypeError: Object of type DocumentField is not JSON serializable

In [15]:
with open(os.path.join(src_path, f'output_{pdf_base}.txt'), 'a', encoding='utf-8') as fout:
    # iterate over tables, lines, and selection marks on each page
    for page in result.pages:
        print("-----------------------------------", file=fout)
        print("\nLines found on page {}".format(page.page_number), file=fout)
        for line in page.lines:
            print("-----------------------------------", file=fout)
            print("...Line '{}'".format(line.content), file=fout)
        for word in page.words:
            print("-----------------------------------", file=fout)
            print(
                "...Word '{}' has a confidence of {}".format(
                    word.content, word.confidence
                )
            , file=fout)
        if page.selection_marks:
            print("-----------------------------------", file=fout)
            for selection_mark in page.selection_marks:
                print(
                    "...Selection mark is '{}' and has a confidence of {}".format(
                        selection_mark.state, selection_mark.confidence
                    )
                , file=fout)

    for i, table in enumerate(result.tables):
        print("-----------------------------------", file=fout)
        print("\nTable {} can be found on page:".format(i + 1), file=fout)
        for region in table.bounding_regions:
            print("...{}".format(i + 1, region.page_number), file=fout)
        for cell in table.cells:
            print(
                "...Cell[{}][{}] has content '{}'".format(
                    cell.row_index, cell.column_index, cell.content
                )
            , file=fout)
    print("-----------------------------------", file=fout)